In [78]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [79]:
class Tableau:
    def __init__(self, A, b, c):
        self.mat = np.hstack([np.vstack([np.zeros(1), b]),
                              np.vstack([c,A])])
        self.m = self.mat.shape[0]
        self.n = self.mat.shape[1]
        self.counter = 0
        
def is_float(element: any) -> bool:
    if element is None: 
        return False
    try:
        float(element)
        return True
    except ValueError:
        return False
    
def read_from_file(inputfile):
    file =  open(inputfile)
    data = file.read().splitlines()
    allowed_signs = {'<=','>='}
    if data[0] == 'min':
        min_max_multiplicator = 1
    elif data[0] == 'max':
        min_max_multiplicator = -1
    c = list(map(float, data[1].split()))
    A = []
    b = []
    for row in data[2:]:
        cur_row = []
        cur_sign = 0 
        for val in row.split():
            if is_float(val):
                cur_row.append(float(val))
            elif val in allowed_signs and cur_sign == 0:
                if val == '<=':
                    cur_sign = 1
                else:
                    cur_sign = -1
        if cur_sign != 0:
            cur_row = np.array(cur_row) * cur_sign
        A.append(cur_row[:-1])
        b.append(cur_row[-1])
    A = np.array(A)
    b = np.array(b).reshape(-1,1)
    c = np.array(c).reshape(1, -1) * min_max_multiplicator
    
    return A, b, c    

In [80]:
def pivot_on(table, row, col):
    epsilon = 1.0e-8
    pivot = table.mat[row][col]
    table.mat[row] /= pivot
    for i in range(table.m):
        if i == row:
            continue
        multiplier = table.mat[i][col]
        table.mat[i] -= multiplier * table.mat[row]

def col_ch(table):
    pivot_col = 1
    lowest = table.mat[0][pivot_col]
    for j in range(1, table.n):
        if table.mat[0][j] < lowest:
            lowest = table.mat[0][j]
            pivot_col = j
    print(f"Минимальное отрицательное значение = {lowest}.\n")
    if lowest >= 0:
        return -1
    return pivot_col

def row_ch(table, pivot_col):
    pivot_row = 0
    min_ratio = -1
    print(f"Частные по ограничениям A[row_i,0]/A[row_i,{pivot_col}] = [", end='')
    for i in range(1, table.m):
        ratio = table.mat[i][0] / table.mat[i][pivot_col]
        print(f"{ratio:8f}, ", end='')
        if ((ratio > 0 and ratio < min_ratio) or min_ratio < 0) and table.mat[i][pivot_col]>0:
            min_ratio = ratio
            pivot_row = i
    print("].\n")
    if min_ratio == -1:
        return -1
    print(f"Найдено целевое значение A[{pivot_row},{pivot_col}], минимальное положительное={min_ratio} в ряду={pivot_row}.\n")
    return pivot_row

def dop_xs(table):
    slack_vars = np.eye(table.m - 1)
    new_mat = np.zeros((table.m, table.n + table.m - 1))
    new_mat[:, :table.n] = table.mat
    new_mat[1:, table.n:] = slack_vars
    table.mat = new_mat
    table.n = table.n + table.m - 1

def base_find(table, col):
    xi = -1
    for i in range(1, table.m):
        if abs(table.mat[i][col] - 1)<epsilon:
            if xi == -1:
                xi = i
            else:
                return -1
        elif not abs(table.mat[i][col] - 0)<epsilon:
            return -1
    return xi

# def print_optimal_vector(table, message):
#     print(f"{message}: ", end='')
#     for j in range(1, table.n):
#         xi = base_find(table, j)
#         if xi != -1:
#             print(f"x{j}={table.mat[xi][0]:8f}, ", end='')
#         else:
#             print(f"x{j}=0, ", end='')
#     print('\n')

def simplex(table):
#Вывод таблицы
    dop_xs(table)
    check_b_positive(table)
    table.counter += 1
    print(f"\n{table.counter}. Базис до преобразования:\n")
    print('.' * 70)
    columns = ['col:','b[i]'] + [f'x{j}' for j in range(1,table.n)]
    data = []
    for i in range(table.m):
        cur_row = []
        if i == 0:
            cur_row.append('max:')
        else:
            cur_row.append(f'b{i}:')
        for j in range(table.n):
            cur_row.append(f'{table.mat[i][j]:8.2f}')
        data.append(cur_row)
    print(pd.DataFrame(data = data, columns = columns))
    print('.' * 70)
    
# Начало итерационного процесса
    k = 0
    while True:
        k += 1
        pivot_col = col_ch(table)
        if pivot_col < 0:
            print(f"Найдено оптимальное значение=A[0,0]={table.mat[0][0]:8f} (нет негативных в ряду 0).\n")
            for j in range(1, table.n):
                xi = base_find(table, j)
                if xi != -1:
                    print(f"x{j}={table.mat[xi][0]:8f}, ", end='')
                else:
                    print(f"x{j}=0, ", end='')
            print('\n')
            break
        print(f"Переменная x{pivot_col} будет базовой, целевая колонка={pivot_col}.\n")

        pivot_row = row_ch(table, pivot_col)
        if pivot_row < 0:
            print("Неограничена (нет целевого ряда).\n")
            break
        print(f"Выбираем переменную x{pivot_row}, Целевая колонка={pivot_row}\n")

        pivot_on(table, pivot_row, pivot_col)
        table.counter += 1
        print(f"\n{table.counter}. Базис после преобразования:\n")
        print('.' * 70)
        columns = ['col:','b[i]'] + [f'x{j}' for j in range(1,table.n)]
        data = []
#Вывод промежуточной таблицы
        for i in range(table.m):
            cur_row = []
            if i == 0:
                cur_row.append('max:')
            else:
                cur_row.append(f'b{i}:')
            for j in range(table.n):
                cur_row.append(f'{table.mat[i][j]:8.2f}')
            data.append(cur_row)
        print(pd.DataFrame(data = data, columns = columns))
        print('.' * 70)
        
        for j in range(1, table.n):
            xi = base_find(table, j)
            if xi != -1:
                print(f"x{j}={table.mat[xi][0]:8f}, ", end='')
            else:
                print(f"x{j}=0, ", end='')
        print('\n')

        if k > 50:
            print(f"Слишком много итераций > {k}.\n")
            break



In [81]:
inputfile = 'test.txt'
A,b,c = read_from_file(inputfile)
simplex(Tableau(A, b, c))


1. Базис до преобразования:

......................................................................
   col:      b[i]        x1        x2        x3        x4        x5        x6  \
0  max:      0.00  -1000.00  -1500.00      0.00      0.00      0.00      0.00   
1   b1:    200.00     20.00     35.00      1.00      0.00      0.00      0.00   
2   b2:   9000.00   1000.00   1500.00      0.00      1.00      0.00      0.00   
3   b3:    800.00     15.00     20.00      0.00      0.00      1.00      0.00   
4   b4:     -0.00     -1.00     -0.00      0.00      0.00      0.00      1.00   
5   b5:     -0.00     -0.00     -1.00      0.00      0.00      0.00      0.00   

         x7  
0      0.00  
1      0.00  
2      0.00  
3      0.00  
4      0.00  
5      1.00  
......................................................................
Минимальное отрицательное значение = -1500.0.

Переменная x2 будет базовой, целевая колонка=2.

Частные по ограничениям A[row_i,0]/A[row_i,2] = [5.714286, 6.00000